In [1]:
deployment = "gpt4"
model = "gpt-4"

# 回顾

In [17]:
from langchain.chains import LLMRequestsChain


template = """Between >>> and <<< are the raw search result text from google.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
  input_variables=["query", "requests_result"],
  template=template,
)


query_chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=PROMPT), output_key="query_info")


In [18]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import SequentialChain

translating_prompt_template = """
translate "{query_info}" into English:

"""

translating_chain = LLMChain(
    llm = llm,
    prompt=PromptTemplate.from_template(translating_prompt_template),
    output_key = "translated"
)




In [23]:
llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)

def overall(question):
    inputs = {
      "query": question,
      "url": "http://www.baidu.com/s?wd=" + question.replace(" ", "+")
    }
    
    overall_chain = SequentialChain(
        chains=[query_chain, translating_chain],
        input_variables=["query","url"],
        output_variables=["translated"],
        verbose=True
    )
    
    return overall_chain(inputs)["translated"]

overall("北京今天天气")



> Entering new SequentialChain chain...

> Finished chain.


'"Updated at 16:51, 32°C, partly cloudy, southeast wind at level 4, temperature ranging from 23~32°C."'

# 没有记忆的模型

In [2]:
import openai
def get_response(input):
    response = openai.ChatCompletion.create(
        engine=deployment, # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are an AI assistant."},   
            {"role": "user", "content": input}
        ],
        temperature = 0.9, 
        max_tokens = 200
      )
    return response.choices[0].message.content

In [3]:
print(get_response("你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？"))
print(get_response("我一般周几健身？"))

作为一款人工智能，我没有具体的日常活动。我的任务就是在这里为你提供帮助，无论是提供信息、答复问题，还是完成你的要求。今天我就在这里为你服务，帮助你管理你的日常事务，如健身计划、娱乐、工作或任何其他需要我的地方。
对不起，作为一个AI助手，我无法获知您的日常活动或习惯。我主要用于解答问题，提供信息和帮助解决问题。如果有任何问题或需要帮助可以向我提问。


# 通过Gradio快速展示功能

In [63]:
!pip install gradio

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [4]:
import gradio as gr
def respond(message, chat_history):
        bot_message = get_response(message)
        chat_history.append((message, bot_message))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #对话框
    msg = gr.Textbox(label="Prompt") #输入框
    btn = gr.Button("Submit") #提交按钮
    #提交
    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) 
gr.close_all()
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


# 提供外部记忆

In [5]:
import openai
def get_response(msg):
    # print(msg)
    response = openai.ChatCompletion.create(
        engine=deployment, # engine = "deployment_name".
        messages=msg,
        temperature = 0.9, 
        max_tokens = 600
    )
    return response.choices[0].message.content

In [6]:
def history_to_prompt(chat_history):
    msg = [{"role": "system", "content": "You are an AI assistant."}]
    i = 0
    for round_trip in chat_history:
        msg.append({"role": "user", "content": round_trip[0]})
        msg.append({"role": "assistant", "content": round_trip[1]})
    return msg

def respond(message, chat_history):
    his_msg = history_to_prompt(chat_history)
    his_msg.append({"role": "user", "content": message})
    bot_message = get_response(his_msg)
    chat_history.append((message, bot_message))
    return "", chat_history



In [8]:
import gradio as gr
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=480) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://98c19ec098e97eb07c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
